# recslam catalog

> Generation of catalog from recslam data


In [16]:
# default_exp tools.recslam.catalog

In [17]:
#| hide
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [18]:
# | export
# | hide

# basic imports
from __future__ import annotations

# sys and paths imports
import json
import os
import sys
from pathlib import Path
from ds_contrib.core.paths import list_paths

# typing imports
from abc import ABC, abstractmethod
from typing import Any, Generic
from ds_contrib.core.utils import T
from dataclasses import dataclass

# cv and image imports
import cv2
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from PIL import Image as PImage

# visualization imports
from ds_contrib.core.plotting import plot

# widgets imports
from IPython.display import display
import ipywidgets as widgets
from ds_contrib.tools.browser import ImageBrowser
from ds_contrib.tools.io.gscloud import GSBrowser
from ds_contrib.core.paths import PathLike
import re
from typing import Literal

from ds_contrib.core.utils import listify
from ds_contrib.core.utils import Iterifiable

In [19]:
# | hide

CWD = Path.cwd()
REPO_DIR = Path(*CWD.parts[: CWD.parts.index("ds_contrib") + 1])
CONFIGS_DIR = REPO_DIR / "configs"
ENV_DIR = CONFIGS_DIR / "env/local"

with open(CONFIGS_DIR / "storage/gscloud/projects_vars.json") as f:
    projects = json.load(f)

# choose project
project = projects["dev"]
env_path = Path(ENV_DIR / f'{project["env"]}_roadly.env')

_ = load_dotenv(env_path)  # read local .env file
google_app_creds = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
roadly_cookie = os.getenv("ROADLY_COOKIE")
print(f"Initial configuration has finished:\nProject: {project}")

Initial configuration has finished:
Project: {'project': 'roadly-project-dev', 'env': 'dev', 'coldline_name': 'standard'}


## RecSLAM item

> Browse and download data from a single RecSLAM folder


In [20]:
from ds_contrib.core.files.structure import FSNode

In [21]:
# | hide

RECSLAM_DATA_PATH = "roadly-dev-standard-videos/Antalya/2022-02-21_13-18-38_4453/"

with open(CONFIGS_DIR / "storage/recslam/recslam_structure.json", "r") as f:
    RECSLAM_STRUCTURE = json.load(f)

In [22]:
fs = FSNode.from_dict(RECSLAM_STRUCTURE)

In [23]:
browser = GSBrowser(project=project["project"], credentials=google_app_creds)
assert browser.is_present(RECSLAM_DATA_PATH), "Data is not present"

In [24]:
remote_files = [Path(p.path).name for p in browser.list(RECSLAM_DATA_PATH)["files"]]

In [25]:
fs.parse_files(remote_files)

set()

In [26]:
fs["common"]

,path,description,type,exists
detections,detections.json,Detections of different distresses and objects...,file,False
device,device.txt,"Device information, camera params, etc.",file,True
gps,gps.csv,GPS information from the device,file,True
heading,heading.csv,Heading (direction) information from the device,file,True
motion,motion.csv,"Motion information from the device, e.g. accel...",file,True
snapshots,snapshots.zip,Snapshots of the video for composition,file,True


In [27]:
files = fs.get_files(only_exists=True)
files

,path,description,type,exists
device,device.txt,"Device information, camera params, etc.",file,True
gps,gps.csv,GPS information from the device,file,True
heading,heading.csv,Heading (direction) information from the device,file,True
motion,motion.csv,"Motion information from the device, e.g. accel...",file,True
snapshots,snapshots.zip,Snapshots of the video for composition,file,True
...,...,...,...,...
video_data_20.data,video_data_20.data,Batch of raw video data,file,True
video_data_18.data,video_data_18.data,Batch of raw video data,file,True
video_data_9.data,video_data_9.data,Batch of raw video data,file,True
video_data_4.data,video_data_4.data,Batch of raw video data,file,True


In [28]:
from bidict import bidict
from ds_contrib.core.paths import Directory
from ds_contrib.tools.io.gscloud import GSBrowserContext

RemotePath = Path
LocalPath = Path


class DownloadableFileStructure:
    def __init__(
        self,
        browser: GSBrowser,
        directory: Directory,
        file_structure: FSNode,
        fs_remote_root: GSBrowserContext,
    ) -> None:
        self._file_structure = file_structure
        self._directory = directory
        self._browser = browser
        self._browser.cd(fs_remote_root)

    def _repr_html_(self):
        return self.df._repr_html_()

    @property
    def df(self) -> pd.DataFrame:
        if hasattr(self, "_df"):
            return self._df
        else:
            files = self._file_structure.get_files(only_exists=True).df
            files["local_path"] = files["path"].apply(
                lambda x: self._directory.path / x if x else None
            )
            files["cached"] = files["local_path"].apply(
                lambda x: x.exists() if x else None
            )
            self._df = files
            return self._df

    def get(self, name: str, download=True) -> Path:
        path = self.df.loc[name, "path"]
        local_path = self.df.loc[name, "local_path"]
        if download and not local_path.exists():
            self._browser.download_file(path, local_path)
        self.df.loc[name, "cached"] = True
        return local_path

In [29]:
dfs = DownloadableFileStructure(browser, Directory(), fs, RECSLAM_DATA_PATH)

In [96]:
dfs

,path,description,type,exists,local_path,cached
device,device.txt,"Device information, camera params, etc.",file,True,/Users/arseniy/Projects/dev/ds_contrib/dev_not...,False
gps,gps.csv,GPS information from the device,file,True,/Users/arseniy/Projects/dev/ds_contrib/dev_not...,False
heading,heading.csv,Heading (direction) information from the device,file,True,/Users/arseniy/Projects/dev/ds_contrib/dev_not...,False
motion,motion.csv,"Motion information from the device, e.g. accel...",file,True,/Users/arseniy/Projects/dev/ds_contrib/dev_not...,False
snapshots,snapshots.zip,Snapshots of the video for composition,file,True,/Users/arseniy/Projects/dev/ds_contrib/dev_not...,False
...,...,...,...,...,...,...
video_data_6.data,video_data_6.data,Batch of raw video data,file,True,/Users/arseniy/Projects/dev/ds_contrib/dev_not...,False
video_data_19.data,video_data_19.data,Batch of raw video data,file,True,/Users/arseniy/Projects/dev/ds_contrib/dev_not...,False
video_data_5.data,video_data_5.data,Batch of raw video data,file,True,/Users/arseniy/Projects/dev/ds_contrib/dev_not...,False
video_data_11.data,video_data_11.data,Batch of raw video data,file,True,/Users/arseniy/Projects/dev/ds_contrib/dev_not...,False


In [103]:
dfs.get("video_data_6.data")

Path('/Users/arseniy/Projects/dev/ds_contrib/dev_notebooks/annotation/tmp2jq8ng4g/video_data_6.data')

In [104]:
dfs

,path,description,type,exists,local_path,cached
device,device.txt,"Device information, camera params, etc.",file,True,/Users/arseniy/Projects/dev/ds_contrib/dev_not...,True
gps,gps.csv,GPS information from the device,file,True,/Users/arseniy/Projects/dev/ds_contrib/dev_not...,False
heading,heading.csv,Heading (direction) information from the device,file,True,/Users/arseniy/Projects/dev/ds_contrib/dev_not...,True
motion,motion.csv,"Motion information from the device, e.g. accel...",file,True,/Users/arseniy/Projects/dev/ds_contrib/dev_not...,False
snapshots,snapshots.zip,Snapshots of the video for composition,file,True,/Users/arseniy/Projects/dev/ds_contrib/dev_not...,False
...,...,...,...,...,...,...
video_data_6.data,video_data_6.data,Batch of raw video data,file,True,/Users/arseniy/Projects/dev/ds_contrib/dev_not...,True
video_data_19.data,video_data_19.data,Batch of raw video data,file,True,/Users/arseniy/Projects/dev/ds_contrib/dev_not...,False
video_data_5.data,video_data_5.data,Batch of raw video data,file,True,/Users/arseniy/Projects/dev/ds_contrib/dev_not...,False
video_data_11.data,video_data_11.data,Batch of raw video data,file,True,/Users/arseniy/Projects/dev/ds_contrib/dev_not...,False


In [105]:
del dfs

In [106]:
dfs

NameError: name 'dfs' is not defined

In [8]:
# | export


@dataclass
class CameraMetadataDTO:
    field_of_view: float
    camera_type: str
    fps: int
    resolution: tuple[int, int]
    focal_length: float


@dataclass
class DeviceMetadataDTO:
    os: str
    device: str
    device_raw: str
    app_version: str
    exposure_mode: str
    max_exposure_time: float
    camera_lense_position: int
    orientation: str
    wide_camera: CameraMetadataDTO
    ultra_wide_camera: CameraMetadataDTO

    @classmethod
    def from_json(cls, json_file):
        with open(json_file) as f:
            data = json.load(f)
            wide_camera = CameraMetadataDTO(
                data["cam_fov_2"],
                data["camera_type_2"],
                data["fps2"],
                tuple(map(int, data["video_resolution_2"].split("x"))),
                data["cam_fx_2"],
            )
            ultra_wide_camera = CameraMetadataDTO(
                data["cam_fov"],
                data["camera_type"],
                data["fps"],
                tuple(map(int, data["video_resolution"].split("x"))),
                data["cam_fx"],
            )
            return cls(
                data["os"],
                data["device"],
                data["device_raw"],
                data["app_version"],
                data["exposure_mode"],
                data["max_exposure"],
                data["cam_lens_pos"],
                data["orientation"],
                wide_camera,
                ultra_wide_camera,
            )

In [9]:
from ds_contrib.core.paths import PathLike
import re


def _read_structure(d: dict, name=None):
    result = {}
    for k, v in d.items():
        if isinstance(v, dict):
            if "data" in v:
                result.update(_read_structure(v["data"], k))
            else:
                key = (name, k if name else k)
                if "path" in v:
                    path = v["path"]
                elif "path_pattern" in v:
                    pattern = v["path_pattern"]
                    path = re.compile(pattern)
                else:
                    raise ValueError(f"No path or path_pattern found in {k}")
                result[key] = {"description": v["desc"], "filename": path}
    return result

In [10]:
def _paths_mapping(paths_index: dict[str, Path], local_path: str | re.Pattern):
    if isinstance(local_path, re.Pattern):
        return [path for name, path in paths_index.items() if local_path.match(name)]
    else:
        path = paths_index.get(local_path)
        return pd.NA if path is None else path

In [59]:
from typing import Literal
from ds_contrib.tools.io.gscloud import GSBrowserContext


class RecslamDTO:
    def __init__(
        self, recslam_path: GSBrowserContext, recslam_structure, browser: GSBrowser
    ) -> None:
        self._recslam_structure = recslam_structure
        self._path = recslam_path
        self._data: pd.DataFrame = None
        self._browser = browser

    def _repr_html_(self):
        if self._data is None:
            return f"{self.__class__.__name__}: No data loaded for {self._path}"
        else:
            return f"{self.__class__.__name__}" + self._data._repr_html_()

    @property
    def structure(self):
        if self._data is None:
            self._read_structure()
        return self._data

    def _read_structure(self):
        files = self._browser.list(self._path)["files"]
        files = [Path(f.path) for f in files]
        paths_index = {path.name: path for path in files}

        structure = pd.DataFrame.from_dict(_read_structure(self._recslam_structure)).T
        structure["source_path"] = structure["filename"].apply(
            lambda x: _paths_mapping(paths_index, x)
        )
        structure.index.set_names(names=["group", "name"], inplace=True)
        structure["local_path"] = pd.NA
        self._data = structure

    def get(self, group: str, name: str, download: bool = False):
        local_path = self.structure.loc[group, name]["local_path"]
        if local_path is pd.NA:
            remote_paths = self.structure.loc[group, name]["source_path"]
            if download:
                local_paths_mapping = self._browser.get_local_paths_mapping(
                    remote_paths
                )
                self._browser.download_files(remote_paths)
                self.structure.loc[(group, name), "local_path"] = list(
                    local_paths_mapping.values()
                )
                return self.structure.loc[group, name]["local_path"]
            else:
                raise ValueError(
                    f"File {group}/{name} not found, try download it previously or set `download`=`True`"
                )
        else:
            return local_path

    def _sample_preset(
        self,
        preset: Literal["basic", "video", "meta", "extended"] = "meta",
        fov: Literal["ultrawide", "wide"] = "wide",
        missing_handling: Literal["drop", "raise"] = "drop",
        existing_handling: Literal["skip", "raise", "overwrite"] = "skip",
    ):
        presets = {
            "video": lambda fov: [(f"camera_{fov}", "video")],
            "meta": lambda fov: [
                (f"camera_{fov}", "timestamps"),
                ("common", "device"),
                ("common", "heading"),
                ("common", "gps"),
                ("common", "motion"),
            ],
        }
        if preset == "basic":
            filtered_structure = self.structure.filter(
                presets["video"](fov) + presets["meta"](fov), axis=0
            )
        elif preset == "extended":
            filtered_structure = self.structure
        elif preset == "video":
            filtered_structure = self.structure.filter(presets["video"](fov), axis=0)
        elif preset == "meta":
            filtered_structure = self.structure.filter(presets["meta"](fov), axis=0)
        else:
            raise ValueError(f"Unknown preset {preset}")

        if missing_handling == "drop":
            filtered_structure["source_path"].dropna()
        elif missing_handling == "raise":
            if filtered_structure["source_path"].isna().any():
                raise ValueError(
                    f"Found missing paths in {filtered_structure[filtered_structure['source_path'].isna()]['filename']}"
                )
        else:
            raise ValueError(f"Unknown missing_handling {missing_handling}")

        if existing_handling == "skip":
            filtered_structure = filtered_structure[
                filtered_structure["local_path"].isna()
            ]
        elif existing_handling == "raise":
            if filtered_structure["local_path"].notna().any():
                raise ValueError(
                    f"Found existing paths in {filtered_structure[filtered_structure['local_path'].notna()]['filename']}"
                )
        elif existing_handling == "overwrite":
            pass
        else:
            raise ValueError(f"Unknown existing_handling {existing_handling}")
        return filtered_structure.index

    def load_preset(
        self,
        preset: Literal["basic", "video", "meta", "extended"] = "meta",
        fov: Literal["ultrawide", "wide"] = "wide",
        missing_handling: Literal["drop", "raise"] = "drop",
        existing_handling: Literal["skip", "raise", "overwrite"] = "skip",
    ):
        """Loads a preset of files from the recslam structure


        Parameters
        ----------
        preset : Literal[&quot;basic&quot;, &quot;video&quot;, &quot;meta&quot;, &quot;extended&quot;], optional
            preset of files to load, all presets are relative to the chosen camera `fov`:
            - basic: video + meta
            - video: only video
            - meta: only meta
            - extended: all files, by default "meta"
        fov : Literal[&quot;ultrawide&quot;, &quot;wide&quot;], optional
            field of view of camera:
            - ultrawide - files without fov suffix
            - wide - files with _2 suffix, by default "wide", main camera
        missing_handling : Literal[&quot;drop&quot;, &quot;raise&quot;], optional
            how to handle missing files:
            - drop - drop missing files
            - raise - raise an error if missing files are found, by default "drop"
        """
        sampled_preset_inds = self._sample_preset(
            preset, fov, missing_handling, existing_handling=existing_handling
        )

        files = []
        for v in self.structure.loc[sampled_preset_inds]["source_path"].to_list():
            if isinstance(v, list):
                for vv in v:
                    files.append(vv)
            else:
                files.append(v)
        if len(files) > 0:
            self._browser.download_files(files)
            self.structure.loc[sampled_preset_inds, "local_path"] = self.structure.loc[
                sampled_preset_inds, "source_path"
            ].apply(
                lambda item: [self._browser.downloads_path / p for p in item]
                if isinstance(item, list)
                else self._browser.downloads_path / item
                if not pd.isna(item)
                else pd.NA
            )

In [60]:
r = RecslamDTO(RECSLAM_DATA_PATH, RECSLAM_STRUCTURE, browser=browser)

In [41]:
r.load_preset("meta", "wide")

File /Users/arseniy/Projects/dev/ds_contrib/dev_notebooks/annotation/tmpx5aozn8y/times_full_2.json already exists, skipping
File /Users/arseniy/Projects/dev/ds_contrib/dev_notebooks/annotation/tmpx5aozn8y/device.txt already exists, skipping
File /Users/arseniy/Projects/dev/ds_contrib/dev_notebooks/annotation/tmpx5aozn8y/heading.csv already exists, skipping
File /Users/arseniy/Projects/dev/ds_contrib/dev_notebooks/annotation/tmpx5aozn8y/gps.csv already exists, skipping
File /Users/arseniy/Projects/dev/ds_contrib/dev_notebooks/annotation/tmpx5aozn8y/motion.csv already exists, skipping


In [31]:
r.structure

description   
group            name                                                                
common           detections      Detections of different distresses and objects...  \
                 device                    Device information, camera params, etc.   
                 gps                               GPS information from the device   
                 heading           Heading (direction) information from the device   
                 motion          Motion information from the device, e.g. accel...   
                 snapshots                  Snapshots of the video for composition   
camera_wide      video           Video captured by the wide camera (main camera...   
                 timestamps       Timestamps of frames captured by the wide camera   
                 timestamps_old  Timestamps of frames captured by the wide came...   
                 raw_data                     Raw data captured by the wide camera   
camera_ultrawide video                     Video captured by the ultrawide camera    
                 timestamps      Timestamps of frames captured by the ultrawide...   
                 timestamps_old  Timestamps of frames captured by the ultrawide...   
                 raw_data                Raw data captured by the ultrawide camera   

                                                                 filename   
group            name                                                       
common           detections                               detections.json  \
                 device                                        device.txt   
                 gps                                              gps.csv   
                 heading                                      heading.csv   
                 motion                                        motion.csv   
                 snapshots                                  snapshots.zip   
camera_wide      video                                            video_2   
                 timestamps                             times_full_2.json   
                 timestamps_old                               times_2.txt   
                 raw_data        re.compile('^video_data_\\d+\\.data_2$')   
camera_ultrawide video                                              video   
                 timestamps                               times_full.json   
                 timestamps_old                                 times.txt   
                 raw_data          re.compile('^video_data_\\d+\\.data$')   

                                                                       source_path   
group            name                                                                
common           detections                                                   <NA>  \
                 device          roadly-dev-standard-videos/Antalya/2022-02-21_...   
                 gps             roadly-dev-standard-videos/Antalya/2022-02-21_...   
                 heading         roadly-dev-standard-videos/Antalya/2022-02-21_...   
                 motion          roadly-dev-standard-videos/Antalya/2022-02-21_...   
                 snapshots       roadly-dev-standard-videos/Antalya/2022-02-21_...   
camera_wide      video           roadly-dev-standard-videos/Antalya/2022-02-21_...   
                 timestamps      roadly-dev-standard-videos/Antalya/2022-02-21_...   
                 timestamps_old  roadly-dev-standard-videos/Antalya/2022-02-21_...   
                 raw_data        [roadly-dev-standard-videos/Antalya/2022-02-21...   
camera_ultrawide video           roadly-dev-standard-videos/Antalya/2022-02-21_...   
                 timestamps      roadly-dev-standard-videos/Antalya/2022-02-21_...   
                 timestamps_old  roadly-dev-standard-videos/Antalya/2022-02-21_...   
                 raw_data        [roadly-dev-standard-videos/Antalya/2022-02-21...   

                                local_path  
group            name                  

In [61]:
r.get("camera_wide", "raw_data", download=True)

File /Users/arseniy/Projects/dev/ds_contrib/dev_notebooks/annotation/tmpx5aozn8y/video_data_0.data_2 already exists, skipping
File /Users/arseniy/Projects/dev/ds_contrib/dev_notebooks/annotation/tmpx5aozn8y/video_data_1.data_2 already exists, skipping
File /Users/arseniy/Projects/dev/ds_contrib/dev_notebooks/annotation/tmpx5aozn8y/video_data_10.data_2 already exists, skipping
File /Users/arseniy/Projects/dev/ds_contrib/dev_notebooks/annotation/tmpx5aozn8y/video_data_11.data_2 already exists, skipping
File /Users/arseniy/Projects/dev/ds_contrib/dev_notebooks/annotation/tmpx5aozn8y/video_data_12.data_2 already exists, skipping
File /Users/arseniy/Projects/dev/ds_contrib/dev_notebooks/annotation/tmpx5aozn8y/video_data_13.data_2 already exists, skipping
File /Users/arseniy/Projects/dev/ds_contrib/dev_notebooks/annotation/tmpx5aozn8y/video_data_14.data_2 already exists, skipping
File /Users/arseniy/Projects/dev/ds_contrib/dev_notebooks/annotation/tmpx5aozn8y/video_data_15.data_2 already exi

ValueError: Must have equal len keys and value when setting with an iterable

In [54]:
r.structure

description   
group            name                                                                
common           detections      Detections of different distresses and objects...  \
                 device                    Device information, camera params, etc.   
                 gps                               GPS information from the device   
                 heading           Heading (direction) information from the device   
                 motion          Motion information from the device, e.g. accel...   
                 snapshots                  Snapshots of the video for composition   
camera_wide      video           Video captured by the wide camera (main camera...   
                 timestamps       Timestamps of frames captured by the wide camera   
                 timestamps_old  Timestamps of frames captured by the wide came...   
                 raw_data                     Raw data captured by the wide camera   
camera_ultrawide video                     Video captured by the ultrawide camera    
                 timestamps      Timestamps of frames captured by the ultrawide...   
                 timestamps_old  Timestamps of frames captured by the ultrawide...   
                 raw_data                Raw data captured by the ultrawide camera   

                                                                 filename   
group            name                                                       
common           detections                               detections.json  \
                 device                                        device.txt   
                 gps                                              gps.csv   
                 heading                                      heading.csv   
                 motion                                        motion.csv   
                 snapshots                                  snapshots.zip   
camera_wide      video                                            video_2   
                 timestamps                             times_full_2.json   
                 timestamps_old                               times_2.txt   
                 raw_data        re.compile('^video_data_\\d+\\.data_2$')   
camera_ultrawide video                                              video   
                 timestamps                               times_full.json   
                 timestamps_old                                 times.txt   
                 raw_data          re.compile('^video_data_\\d+\\.data$')   

                                                                       source_path   
group            name                                                                
common           detections                                                   <NA>  \
                 device          roadly-dev-standard-videos/Antalya/2022-02-21_...   
                 gps             roadly-dev-standard-videos/Antalya/2022-02-21_...   
                 heading         roadly-dev-standard-videos/Antalya/2022-02-21_...   
                 motion          roadly-dev-standard-videos/Antalya/2022-02-21_...   
                 snapshots       roadly-dev-standard-videos/Antalya/2022-02-21_...   
camera_wide      video           roadly-dev-standard-videos/Antalya/2022-02-21_...   
                 timestamps      roadly-dev-standard-videos/Antalya/2022-02-21_...   
                 timestamps_old  roadly-dev-standard-videos/Antalya/2022-02-21_...   
                 raw_data        [roadly-dev-standard-videos/Antalya/2022-02-21...   
camera_ultrawide video           roadly-dev-standard-videos/Antalya/2022-02-21_...   
                 timestamps      roadly-dev-standard-videos/Antalya/2022-02-21_...   
                 timestamps_old  roadly-dev-standard-videos/Antalya/2022-02-21_...   
                 raw_data        [roadly-dev-standard-videos/Antalya/2022-02-21...   

                                local_path  
group            name                  

In [89]:
r.structure.loc[("common", "detections")]

Index(['description', 'filename', 'source_path', 'local_path'], dtype='object')

In [92]:
r.structure.loc[("common", "detections"), "local_path"] = [5, 6]

ValueError: Must have equal len keys and value when setting with an iterable

In [38]:
r.structure.loc["camera_wide", "raw_data"]["source_path"]

[Path('roadly-dev-standard-videos/Antalya/2022-02-21_13-18-38_4453/video_data_0.data_2'),
 Path('roadly-dev-standard-videos/Antalya/2022-02-21_13-18-38_4453/video_data_1.data_2'),
 Path('roadly-dev-standard-videos/Antalya/2022-02-21_13-18-38_4453/video_data_10.data_2'),
 Path('roadly-dev-standard-videos/Antalya/2022-02-21_13-18-38_4453/video_data_11.data_2'),
 Path('roadly-dev-standard-videos/Antalya/2022-02-21_13-18-38_4453/video_data_12.data_2'),
 Path('roadly-dev-standard-videos/Antalya/2022-02-21_13-18-38_4453/video_data_13.data_2'),
 Path('roadly-dev-standard-videos/Antalya/2022-02-21_13-18-38_4453/video_data_14.data_2'),
 Path('roadly-dev-standard-videos/Antalya/2022-02-21_13-18-38_4453/video_data_15.data_2'),
 Path('roadly-dev-standard-videos/Antalya/2022-02-21_13-18-38_4453/video_data_16.data_2'),
 Path('roadly-dev-standard-videos/Antalya/2022-02-21_13-18-38_4453/video_data_17.data_2'),
 Path('roadly-dev-standard-videos/Antalya/2022-02-21_13-18-38_4453/video_data_18.data_2'),
 

In [27]:
r.structure.loc["camera_wide"]["local_path"]

name
video             /Users/arseniy/Projects/dev/ds_contrib/dev_not...
timestamps        /Users/arseniy/Projects/dev/ds_contrib/dev_not...
timestamps_old                                                 <NA>
raw_data                                                       <NA>
Name: local_path, dtype: object

In [323]:
browser.downloads_dir.cleanup()

In [ ]:
from pydantic import BaseModel


class VideoDataDTO(BaseModel):
    video: Path | None = None
    timestamps: Path | None = None
    timestamps_old: Path | None = None
    raw_data: list[Path] | None = None

    class Config:
        version = "1.0.0"


class RecslamDTO(BaseModel):
    device: Path
    gps: Path
    imu: Path
    heading: Path
    ultrawide_video: VideoDataDTO | None = None
    wide_video: VideoDataDTO | None = None
    snapshots: Path | None = None

    class Config:
        version = "1.0.0"

    def from_list(self, paths: list[Path]):
        ultra_wide_video = VideoDataDTO()
        wide_video = VideoDataDTO()
        for p in paths:
            match p.name:
                case "device.txt":
                    self.device = p
                case "gps.csv":
                    self.gps = p
                case "motion.csv":
                    self.imu = p
                case "heading.csv":
                    self.heading = p
                case "snapshots.zip":
                    self.snapshots = p
                case "video":
                    ultra_wide_video.video = p
                case "video_2":
                    wide_video.video = p
                case _:
                    if p.name.startswith("video_data"):
                        if p.name.endswith("_2"):
                            self.wide_video.raw_data.append(p)
                        else:
                            self.ultrawide_video.raw_data.append(p)
                    elif p.name.startswith("times"):
                        if p.name.startswith("times_full"):
                            if p.stem.endswith("_2"):
                                self.wide_video.timestamps = p
                            else:
                                self.ultrawide_video.timestamps = p
                        else:
                            if p.stem.endswith("_2"):
                                self.wide_video.timestamps_old = p
                            else:
                                self.ultrawide_video.timestamps_old = p